In [32]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
import numpy as np
#from sklearn.pipeline import Pipeline
from arch import arch_model

In [26]:
def load_data(symbol="SPY", start="1994-01-01", end="2021-01-01", interval='1d'):
    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)
    return df

def preprocess_data(df):
    df = df[['Date', 'Close']]
    df.dropna(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def train_test(list_df, train_size=0.7, test_size=0.3):
    result = []
    for df in list_df:
        X = df.index.values.reshape(-1, 1)  
        y = df['Close'].values              

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=train_size, test_size=test_size, shuffle=False
        )
        result.append([X_train, X_test, y_train, y_test])
    return result

def etl_pipeline(train_size=0.7, test_size=0.3):
    gspc = load_data('^GSPC')
    dji = load_data('^DJI')
    nyse = load_data('^NYA')
    
    gspc_close = preprocess_data(gspc)
    dji_close = preprocess_data(dji)
    nyse_close = preprocess_data(nyse)

    list_df_close = [gspc_close, dji_close, nyse_close]

    splits = train_test(list_df_close, train_size=train_size, test_size=test_size)

    return {
        "GSPC": {"X_train": splits[0][0], "X_test": splits[0][1], "y_train": splits[0][2], "y_test": splits[0][3]},
        "DJI":  {"X_train": splits[1][0], "X_test": splits[1][1], "y_train": splits[1][2], "y_test": splits[1][3]},
        "NYSE": {"X_train": splits[2][0], "X_test": splits[2][1], "y_train": splits[2][2], "y_test": splits[2][3]},
    }



In [16]:
gspc=load_data('^GSPC')
dji=load_data('^DJI')
nyse=load_data('^NYA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
gspc_close=preprocess_data(gspc)
dji_close=preprocess_data(dji)
nyse_close=preprocess_data(nyse)

C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3228835240.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)  # Remove any missing values
C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3228835240.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3228835240.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [21]:
print(gspc_close)

Price             Close
Ticker            ^GSPC
Date                   
1994-01-03   465.440002
1994-01-04   466.890015
1994-01-05   467.549988
1994-01-06   467.119995
1994-01-07   469.899994
...                 ...
2020-12-24  3703.060059
2020-12-28  3735.360107
2020-12-29  3727.040039
2020-12-30  3732.040039
2020-12-31  3756.070068

[6799 rows x 1 columns]


In [27]:
# Run the ETL pipeline
result = etl_pipeline()

# Assign GSPC values
X_gspc_train = result["GSPC"]["X_train"]
X_gspc_test  = result["GSPC"]["X_test"]
y_gspc_train = result["GSPC"]["y_train"]
y_gspc_test  = result["GSPC"]["y_test"]

# Assign DJI values
X_dji_train = result["DJI"]["X_train"]
X_dji_test  = result["DJI"]["X_test"]
y_dji_train = result["DJI"]["y_train"]
y_dji_test  = result["DJI"]["y_test"]

# Assign NYSE values
X_nyse_train = result["NYSE"]["X_train"]
X_nyse_test  = result["NYSE"]["X_test"]
y_nyse_train = result["NYSE"]["y_train"]
y_nyse_test  = result["NYSE"]["y_test"]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3814708522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3814708522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\idris\AppData\Local\Temp\ipykernel_20120\3814708522.py:8: Setti

In [31]:
print(X_dji_train)

[['1994-01-03T00:00:00.000000000']
 ['1994-01-04T00:00:00.000000000']
 ['1994-01-05T00:00:00.000000000']
 ...
 ['2012-11-20T00:00:00.000000000']
 ['2012-11-21T00:00:00.000000000']
 ['2012-11-23T00:00:00.000000000']]


In [ ]:

# Function to calculate log returns
def calculate_log_returns(prices):
    return np.log(prices / prices.shift(1)).dropna()

# Function to predict average and volatility using GARCH(1,1)
def garch_predictions(prices, forecast_horizon=1):
    # Calculate daily log returns for the past 90 days
    log_returns = calculate_log_returns(prices).tail(90)
    
    # Fit the GARCH(1,1) model with constant mean
    model = arch_model(log_returns, mean='constant', vol='GARCH', p=1, q=1)
    model_fit = model.fit(disp='off')

    # Forecast the next 'forecast_horizon' days
    forecast = model_fit.forecast(horizon=forecast_horizon)
    
    # Extract average (mean) and volatility (variance) forecast
    predicted_mean = forecast.mean.iloc[-1].values[0]
    predicted_volatility = np.sqrt(forecast.variance.iloc[-1].values[0])  # Convert variance to standard deviation
    
    return predicted_mean, predicted_volatility

